In [17]:
def init_game(str_heroes):
    """
    Heroes:
    
    llynne = "Lady Lynne"
    """
    
    
    heroes = []
    for hero in str_heroes:
        heroes.append(Hero(hero))

In [18]:
class Hero():
    def __init__(self, name):
        self.name = name
        
        #Heroes Names:
        self.llynne = "Lady Lynne"
        
        #Initialize Heroes:
        self.init_heroes(name)
        
    def init_heroes(self, name):
        if name == self.llynne:
            hp = 100
            ap = 100
            df = 0
            at = 2
        
        print("Hero", name, "was initialized.")

In [19]:
#Test game

init_game(["Lady Lynne"])

Hero Lady Lynne was initialized.
